In [1]:
from mingpt.utils import set_seed
import numpy as np
import scipy as sp
import torch
import torch.nn as nn
from torch.nn import functional as F
import math
from torch.utils.data import Dataset
from mingpt.model import GPT
from mingpt.trainer import Trainer
from mingpt.utils import set_seed, setup_logging, CfgNode as CN

import routegym.env
import networkx as nx

In [2]:
obs = []
actions = []
returns = [0]
done_idxs = []
stepwise_returns = []

transitions_per_buffer = np.zeros(50, dtype=int)
num_trajectories = 0

In [3]:
node_number = 12

env = None
while env is None:
    try:
        A = sp.sparse.random(node_number, node_number, density=0.1, format='csr')
        A.data[:] = 1
        A = A.todense()
        A = np.ma.array(A, mask=np.eye(node_number)).filled(fill_value=0).astype(int)
        print("sparsity = %.2f" % (1 - np.sum(A)/A.size))
        env = routegym.env.ShortestRouteEnv(nx.from_numpy_array(A), 0, 5, random_weights=(1,10))
    except:
        pass
# env.render()
print(A)
print(env.graph.adj_mat)
env.get_dijkstra()

sparsity = 0.90
[[0 0 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 1 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 1]
 [0 1 0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [1 1 0 0 0 0 0 0 0 0 0 0]]
[[-1 -1  1 -1 -1 -1 -1 -1  1 -1 -1  1]
 [-1 -1 -1 -1 -1  1  1 -1 -1  1 -1  1]
 [ 1 -1 -1  1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1  1 -1 -1 -1 -1  1 -1 -1  1 -1]
 [-1 -1 -1 -1 -1  1 -1 -1 -1 -1 -1  1]
 [-1  1 -1 -1  1 -1  1 -1  1 -1 -1 -1]
 [-1  1 -1 -1 -1  1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1  1 -1 -1 -1 -1 -1 -1 -1 -1]
 [ 1 -1 -1 -1 -1  1 -1 -1 -1 -1 -1 -1]
 [-1  1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1  1 -1 -1 -1 -1 -1 -1 -1 -1]
 [ 1  1 -1 -1  1 -1 -1 -1 -1 -1 -1 -1]]


([0, 8, 5], 2)

In [7]:
env.graph.adjacent_indices[2]

array([0, 3])

In [8]:
env.step(1)

1 not in [0 3]
INVALID!


(2, -1000.0, False, {})

In [14]:
position, reward, done, _ = env.step(3)

In [17]:
print(position)

3
